In [1]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [3]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [4]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [8]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [9]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

<Response [200]>

In [ ]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json() # uncomment to see error

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [10]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

Failed to get result from API due to error:
Invalid control character '\r' at: line 1 column 289 (char 288)


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [11]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"status":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2091\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 1781\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 628\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 179"}'

'{"status":200,"result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [12]:
# your code here
def doGrade(grade):
    link = 'https://www.nationsreportcard.gov/''Dataservice/GetAdhocData.aspx?''type=data&subject=writing&grade=' + str(grade) + '&''subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'
    naep_resp1 = requests.get(link)
    print(naep_resp1)
    print(type(naep_resp1))
    try:
        results = naep_resp1.json()['result']
        return results
    except Exception as e:
        print('Failed to get result from API due to error:')
        print(e) # or just: pass
doGrade(4)
doGrade(8)
doGrade(12)

<Response [200]>
<class 'requests.models.Response'>
Failed to get result from API due to error:
Invalid control character '\r' at: line 1 column 289 (char 288)
<Response [200]>
<class 'requests.models.Response'>


[{'year': 2011,
  'sample': 'R3',
  'yearSampleLabel': '2011',
  'Cohort': 2,
  'CohortLabel': 'Grade 8',
  'stattype': 'MN:MN',
  'subject': 'WRI',
  'grade': 8,
  'scale': 'WRIRP',
  'jurisdiction': 'NP',
  'jurisLabel': 'National public',
  'variable': 'GENDER',
  'variableLabel': 'Gender',
  'varValue': '1',
  'varValueLabel': 'Male',
  'value': 139.099504632971,
  'isStatDisplayable': 1,
  'errorFlag': 0},
 {'year': 2011,
  'sample': 'R3',
  'yearSampleLabel': '2011',
  'Cohort': 2,
  'CohortLabel': 'Grade 8',
  'stattype': 'MN:MN',
  'subject': 'WRI',
  'grade': 8,
  'scale': 'WRIRP',
  'jurisdiction': 'NP',
  'jurisLabel': 'National public',
  'variable': 'GENDER',
  'variableLabel': 'Gender',
  'varValue': '2',
  'varValueLabel': 'Female',
  'value': 158.567104984955,
  'isStatDisplayable': 1,
  'errorFlag': 0}]

<Response [200]>
<class 'requests.models.Response'>


[{'year': 2011,
  'sample': 'R3',
  'yearSampleLabel': '2011',
  'Cohort': 3,
  'CohortLabel': 'Grade 12',
  'stattype': 'MN:MN',
  'subject': 'WRI',
  'grade': 12,
  'scale': 'WRIRP',
  'jurisdiction': 'NP',
  'jurisLabel': 'National public',
  'variable': 'GENDER',
  'variableLabel': 'Gender',
  'varValue': '1',
  'varValueLabel': 'Male',
  'value': 141.256977963264,
  'isStatDisplayable': 1,
  'errorFlag': 0},
 {'year': 2011,
  'sample': 'R3',
  'yearSampleLabel': '2011',
  'Cohort': 3,
  'CohortLabel': 'Grade 12',
  'stattype': 'MN:MN',
  'subject': 'WRI',
  'grade': 12,
  'scale': 'WRIRP',
  'jurisdiction': 'NP',
  'jurisLabel': 'National public',
  'variable': 'GENDER',
  'variableLabel': 'Gender',
  'varValue': '2',
  'varValueLabel': 'Female',
  'value': 155.385916780351,
  'isStatDisplayable': 1,
  'errorFlag': 0}]

# 2. Example 2: needs credentials; no wrapper

Create an account here: https://www.yelp.com/developers/v3/manage_app

In [13]:
## get the key
API_KEY = "ln1hCQ7rMo-u-ZgozN7B1Wo0wFFXq4gxMv3cbmqqpGevxDnXVbZ-_xC_2mOjkxQvNoUS3Ll8QNrh7cwR7BoroHzf6ou1mDAV-9_cc1Zj7O6-6Kwg4d6XQT7I3KszZnYx"

In [17]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Hanover,NH,03755"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()


<Response [200]>

In [15]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': 'neBEWvgHNhsXIBImCYx_6A',
 'alias': 'murphy-s-on-the-green-hanover',
 'name': 'Murphy’s on the Green',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/76DCQrgDN8Nqe3QKDA-opw/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/murphy-s-on-the-green-hanover?adjust_creative=713booav9u8BVC_uTkpQmg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=713booav9u8BVC_uTkpQmg',
 'review_count': 176,
 'categories': [{'alias': 'newamerican', 'title': 'New American'},
  {'alias': 'seafood', 'title': 'Seafood'},
  {'alias': 'pubs', 'title': 'Pubs'}],
 'rating': 3.7,
 'coordinates': {'latitude': 43.702, 'longitude': -72.2894949},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '11 S Main St',
  'address2': '',
  'address3': '',
  'city': 'Hanover',
  'zip_code': '03755',
  'country': 'US',
  'state': 'NH',
  'display_address': ['11 S Main St', 'Hanover, NH 03755']},
 'phone': '+16036434075',
 'display_phone': '(603) 643-4075',
 'dis

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,attributes
0,neBEWvgHNhsXIBImCYx_6A,murphy-s-on-the-green-hanover,Murphy’s on the Green,https://s3-media2.fl.yelpcdn.com/bphoto/76DCQr...,False,https://www.yelp.com/biz/murphy-s-on-the-green...,176,"[{'alias': 'newamerican', 'title': 'New Americ...",3.7,"{'latitude': 43.702, 'longitude': -72.2894949}",[delivery],$$,"{'address1': '11 S Main St', 'address2': '', '...",+16036434075,(603) 643-4075,295.207523,"{'business_temp_closed': None, 'menu_url': Non..."
1,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,False,https://www.yelp.com/biz/mollys-restaurant-and...,509,"[{'alias': 'tradamerican', 'title': 'American'...",3.9,"{'latitude': 43.701144, 'longitude': -72.2894249}",[delivery],$$,"{'address1': '43 South Main St', 'address2': '...",+16036432570,(603) 643-2570,250.830160,"{'business_temp_closed': None, 'menu_url': 'ht..."
2,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,False,https://www.yelp.com/biz/base-camp-cafe-hanove...,247,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...",4.4,"{'latitude': 43.700626, 'longitude': -72.2887803}",[delivery],$$,"{'address1': '3 Lebanon St', 'address2': 'Ste ...",+16036432007,(603) 643-2007,196.139758,"{'business_temp_closed': None, 'menu_url': 'ht..."
3,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,False,https://www.yelp.com/biz/sawtooth-kitchen-hano...,25,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.1,"{'latitude': 43.70158, 'longitude': -72.289641}",[],NaN,"{'address1': '33 S Main St', 'address2': '', '...",+16036435134,(603) 643-5134,242.607552,"{'business_temp_closed': None, 'menu_url': 'ht..."
4,34j_2nRCVQBTKafJvncZlg,pine-restaurant-hanover-2,PINE Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/c2SgUR...,False,https://www.yelp.com/biz/pine-restaurant-hanov...,226,"[{'alias': 'newamerican', 'title': 'New Americ...",3.6,"{'latitude': 43.7024572, 'longitude': -72.2875...",[delivery],$$$,"{'address1': '2 E Wheelock St', 'address2': No...",+16036468000,(603) 646-8000,256.004291,"{'business_temp_closed': None, 'menu_url': 'ht..."


In [16]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,price,phone,display_phone
neBEWvgHNhsXIBImCYx_6A,neBEWvgHNhsXIBImCYx_6A,murphy-s-on-the-green-hanover,Murphy’s on the Green,https://s3-media2.fl.yelpcdn.com/bphoto/76DCQr...,https://www.yelp.com/biz/murphy-s-on-the-green...,$$,+16036434075,(603) 643-4075
8ybF6YyRldtZmU9jil4xlg,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,https://www.yelp.com/biz/mollys-restaurant-and...,$$,+16036432570,(603) 643-2570
XVGEEIH5rVB2QzW-qywcJw,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,https://www.yelp.com/biz/base-camp-cafe-hanove...,$$,+16036432007,(603) 643-2007
5WW4g_LRwau29KyjZGLyAA,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,https://www.yelp.com/biz/sawtooth-kitchen-hano...,NaN,+16036435134,(603) 643-5134
34j_2nRCVQBTKafJvncZlg,34j_2nRCVQBTKafJvncZlg,pine-restaurant-hanover-2,PINE Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/c2SgUR...,https://www.yelp.com/biz/pine-restaurant-hanov...,$$$,+16036468000,(603) 646-8000
jAFLKiVXq9z_h4ZPy0nLMA,jAFLKiVXq9z_h4ZPy0nLMA,the-works-café-hanover,The Works Café,https://s3-media2.fl.yelpcdn.com/bphoto/olpsy-...,https://www.yelp.com/biz/the-works-caf%C3%A9-h...,NaN,+16032772082,(603) 277-2082
KA8yhrd-ClVYMyOefXdVYg,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,https://www.yelp.com/biz/lous-restaurant-and-b...,$$,+16036433321,(603) 643-3321


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://docs.developer.yelp.com/reference/v3_business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [38]:
# your code here
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Higganum,CT,06441"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)


## then, look at structure of response
yelp_genjson = yelp_genresp.json()

## example business
# yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
# yelp_gendf


In [39]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

# yelp_stronly_df


In [41]:
def get_business_reviews(business_ids):
    base_url = "https://api.yelp.com/v3/businesses/{}/reviews"
    header = {'Authorization': f"Bearer {API_KEY}"}
    total_reviews = []
    for bus_id in business_ids:
        total_query = base_url.format(bus_id)
        yelp_genresp = requests.get(total_query, headers = header)
        yelp_genresp
        yelp_genjson = yelp_genresp.json()['reviews']
        yelp_genjson
        for rev_dict in yelp_genjson:
            rev_dict["business_id"] = bus_id
        total_reviews += yelp_genjson
    final_df = pd.DataFrame(total_reviews)
    return final_df

get_business_reviews(["MRTdhLfHe-Uw-qQnQXxgVg", "PqyJx0ndtwBT9wLeFYnhoQ", "zeFASIGTjBwwPNf0ab86Jw"])

KeyError: 'reviews'